<a href="https://colab.research.google.com/github/dmalch/genealogy-research/blob/main/GenealogyResearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Шаг 1. Открыть таблицы с индексированными данными из метрических книг.

In [6]:
import pandas as pd
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

# Open the spreadsheet
sh = gc.open('Кириллово общий индекс')

# Select the worksheet
births = sh.worksheet('Рождения')

# Get all values from the worksheet
birthsData = births.get_all_values()

# Convert data to pandas DataFrame
births_df = pd.DataFrame(birthsData[1:], columns=birthsData[0])

# Now you can work with the DataFrame 'df'
print(births_df.head())


  Номер муж Номер жен        Дата           Место                      Имя  \
0         1            01.01.1822     с. Кирилово      Ефим Спиридонов [1]   
1         2            01.01.1822     с. Кирилово      Василий Сидоров [5]   
2                   1  01.01.1822     с. Кирилово  Настасия Харитонова [6]   
3         3            02.01.1822  д. Гоголев Бор       Василий Иванов [7]   
4         4            02.01.1822  д. Гоголев Бор       Исаия Филиппов [8]   

       Имя в документе                    Отец  Отец в документе Мать  \
0      Ефим Спиридонов      Спиридон Фомин [2]    Спиридон Фомин        
1      Василий Сидоров    Сидор Ермолаев [235]    Сидор Ермолаев        
2  Настасия Харитонова  Харитон Панфенов [236]  Харитон Панфенов        
3       Василий Иванов        Иван Иванов [69]       Иван Иванов        
4        Исаия Филипов   Василий Исаевич [238]   Василий Исаевич        

  Мать в документе         Воспреемник 1 Воспреемник 1 в документе  \
0                     

In [17]:
# Extracting a subset of columns
children_df = births_df[['Имя в документе', 'Место']]
fathers_df = births_df[['Отец в документе', 'Место']]
mothers_df = births_df[['Мать в документе', 'Место']]
godparent1_df = births_df[['Воспреемник 1 в документе', 'Место']]
godparent2_df = births_df[['Воспреемник 2 в документе', 'Место']]

children_df.columns = ['Name', 'Place']
fathers_df.columns = ['Name', 'Place']
mothers_df.columns = ['Name', 'Place']
godparent1_df.columns = ['Name', 'Place']
godparent2_df.columns = ['Name', 'Place']

combined_df = pd.concat([children_df, fathers_df, mothers_df, godparent1_df, godparent2_df], ignore_index=True)

# Displaying the new DataFrames
# print(children_df.head())
# print(fathers_df.head())
# print(mothers_df.head())
# print(godparent1_df.head())
# print(godparent2_df.head())
print(combined_df.head())
print(combined_df.tail())
print(combined_df.info())
print(combined_df.describe())

                  Name           Place
0      Ефим Спиридонов     с. Кирилово
1      Василий Сидоров     с. Кирилово
2  Настасия Харитонова     с. Кирилово
3       Василий Иванов  д. Гоголев Бор
4        Исаия Филипов  д. Гоголев Бор
                     Name         Place
1485        Ирина Аникина  с. Кирилово 
1486  Анисья Савастьянова  с. Кирилово 
1487     Евдокия Семенова  с. Кирилово 
1488       Марья Миронова  с. Кирилово 
1489      Анна Григорьева  с. Кирилово 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1490 entries, 0 to 1489
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Name    1490 non-null   object
 1   Place   1490 non-null   object
dtypes: object(2)
memory usage: 23.4+ KB
None
        Name        Place
count   1490         1490
unique  1005            4
top           с. Кирилово
freq     297         1045
